In [1]:
# Imported Libraries
import numpy as np # Used for array.
import time # Used to keep track of running time.

In [2]:
# Goal state and imported puzzles from project description
goal=np.array(
    [[1,2,3],
    [4,5,6],
    [7,8,0]])

depth_0=np.array(
    [[1,2,3],
    [4,5,6],
    [7,8,0]])

depth_2=np.array(
    [[1,2,3],
    [4,5,6],
    [0,7,8]])

depth_4=np.array(
    [[1,2,3],
    [5,0,6],
    [4,7,8]])

depth_8=np.array(
    [[1,3,6],
    [5,0,2],
    [4,7,8]])

depth_12=np.array(
    [[1,3,6],
    [5,0,7],
    [4,8,2]])

depth_16=np.array(
    [[1,6,7],
    [5,0,3],
    [4,8,2]])

depth_20=np.array(
    [[7,1,2],
    [4,8,5],
    [6,3,0]])

depth_24=np.array(
    [[0,7,2],
    [4,6,1],
    [3,5,8]])

In [3]:
# Main node class for each node on the tree
class Node:
    def __init__(self, board):
        #Initialize variables
        self.board = board
        self.parent = None
        self.depth = 0
        self.expanded = 0
        self.max_queue_size = 0
        self.cost = 0
        
    def getBoard(self):
        return self.board
    
    def isGoal(self,goal): # Are you my goal state?
        if(np.array_equal(self.board, goal)):
            return True # Return True if yes
        else:
            return False # Else False and keep looking
        
    def getChild(self):
        child = [] # Used to store generated child states from parent
        # Find the position of 0 in the board, and set x,y to that position
        for i in range(len(self.board)):
            for j in range(len(self.board[i])):
                if self.board[i][j] == 0:
                    x, y = i, j
                    break

        # Helper function to swap two numbers in the array
        def swap(x1, y1, x2, y2):
            temp = self.board.copy()
            temp[x1][y1], temp[x2][y2] = temp[x2][y2], temp[x1][y1]
            return temp

        # Generate child states by swapping 0 with neighboring numbers
        neighbors = [(x+1, y), (x-1, y), (x, y+1), (x, y-1)] # The 4 operators: Up, Down, Right, Left
        for i, j in neighbors:
            if 0 <= i < len(self.board) and 0 <= j < len(self.board[x]): # Check if the 4 Operator is within range of the board
                newChild = Node(swap(x, y, i, j)) # Generate a new child for each of the valid operators.
                newChild.parent = self # Set current node as parent to the generated childrens.
                newChild.depth = self.depth + 1 # Add 1 to the depth of the childrens.
                child.append(newChild) # Store the childrens in the child array.

        return child

In [4]:
# Helper functions

# Misplaced Tile function
def misplaced_tiles(board, goal):
    misplaced_count = 0 #track of # of misplaced tiles
    #iterate through the board and see if any of the numbers are out of place that are not 0, if they are -> add one to the counter.
    for i in range(len(board)):
        for j in range(len(board[i])):
            if board[i][j] != goal[i][j] and board[i][j] != 0:
                misplaced_count+=1
    return misplaced_count

# Manhattan distance formula for the position of misplaced tiles.
def manhattan_distance(board, goal):
    distance = 0
    goal_dict = {} 
    # Create a dictionary for the goal state.
    for x1 in range(len(goal)):
        for y1 in range(len(goal[x1])):
            goal_dict[goal[x1][y1]] = (x1, y1)

    # Check the coordinate of the number using the dictionary
    for x2 in range(len(board)):
        for y2 in range(len(board[x1])):
            if board[x2][y2] != 0:
                x, y = goal_dict[board[x2][y2]]
                distance += abs(x - x2) + abs(y - y2) # Return the Manhattan distance for the board

    return distance

# Print board
def printBoard(board):
        for row in board:
            print(row)

In [5]:
# Search Algorithms, combined into one as each are just A* with different cost function.

def search(board, goal, heuristic):
    # Return the board if it is already solved.
    if board.isGoal(goal):
        return board
    
    queue = [] # Initate a queue as a working queue to store the states that needed to be checked
    repeated_states = [] # Store already processed states
    expansion_count = 0  # Keep track of expanded nodes count
    max_queue_size = len(queue) # Keep track of max queue size
    
    queue.append(board) # Add board to the working queue
    while(len(queue) != 0):
        current = queue.pop(0) # Take the top element and use it as the current state being checked
        
        if current.isGoal(goal): # Check are you my goal state. If yes -> return the board state. If No-> get its child states
            current.max_queue_size = max_queue_size # Update the current node's max queue size
            return current
        
        repeated_states.append(current.getBoard().tolist()) # Add the board to list of already seen board states.
        
        child = current.getChild() #Get the child board states of the current board state, add 1 to expansion size
        expansion_count += 1
        
        for i in child:
            if i.getBoard().tolist() not in repeated_states: #if the board is not seen before
                if(heuristic == 2):
                    i.cost = misplaced_tiles(i.getBoard(), goal) + i.depth # If misplaced tile heuristic is selected, we set the cost to the misplaced tile function + leaf depth
                if(heuristic == 3):
                    i.cost = manhattan_distance(i.getBoard(), goal) + i.depth # If manhattan heuristic is selected, we set the cost to the manhattan distance + leaf depth
                i.expanded = expansion_count # Keep track of how many nodes are expanded
                queue.append(i) # Add the child to the working queue
                
        max_queue_size = max(len(queue), max_queue_size) # Update the max queue size of the search
        
        if(heuristic == 2 or heuristic == 3): # If the misplaced tile or manhattan distance is used
            queue = sorted(queue, key=lambda x: x.cost) # Sort the working queue according to cost to expand node.

In [8]:
# Driver Code
def main():
    print("Welcome to an 8-Puzzle Solver. Type '1' to use a default puzzle Type '2' to create your own.")
    choice = int(input())
    
    if choice == 1:
        print("You wish to use the default puzzle.\n" +
            "Type '1' for Depth 0 \n" +
            "Type '2' for Depth 2 \n" +
            "Type '3' for Depth 4 \n" +
            "Type '4' for Depth 8 \n" +
            "Type '5' for Depth 16 \n"+
            "Type '6' for Depth 20 \n"+
            "Type '7' for Depth 24. \n")
        puzchoice = int(input())
        if puzchoice == 1:
            print("You selected Depth 0")
            puzzle = depth_0
        elif puzchoice == 2:
            print("You selected Depth 2")
            puzzle = depth_2
        elif puzchoice == 3:
            print("You selected Depth 4")
            puzzle = depth_4
        elif puzchoice == 4:
            print("You selected Depth 8")
            puzzle = depth_8
        elif puzchoice == 5:
            print("You selected Depth 12")
            puzzle = depth_12
        elif puzchoice == 6:
            print("You selected Depth 16")
            puzzle = depth_16
        elif puzchoice == 7:
            print("You selected Depth 20")
            puzzle = depth_20
        elif puzchoice == 8:
            print("You selected Depth 24")
            puzzle = depth_24
        else:
            print("Invalid selection, program will now exit.")
            return 0
        
        printBoard(puzzle)
        
    if choice == 2:
        print("\nYou select to create your own puzzle. Use a 0 to represent the blank. Please enter a valid 8-puzzles. And please delimit the number using space. Press enter when you finish entering a row.")
        print("Enter the first row: ")
        row1 = input()
        print("Enter the second row: ")
        row2 = input()
        print("Enter the third row: ")
        row3 = input()
        
        puzzle = np.array(
                    [[int(row1[0]),int(row1[2]),int(row1[4])],
                    [int(row2[0]),int(row2[2]),int(row2[4])],
                    [int(row3[0]),int(row3[2]),int(row3[4])]])

    board = Node(puzzle)
    
    print("Enter your choice of algorithm: Type '1' for Uniform Cost Search. Type '2' for A* with the Misplaced Tile heuristic. Type '3' for A* with the Manhattan distance heuristic.")
    choice = int(input())
    print()
        
    if choice >= 1 and choice <= 3:
        if choice == 1:
            print("You selected Uniform Cost Search")
        if choice == 2:
            print("You selected A* with the Misplaced Tile heuristic")
        if choice == 3:
            print("You selected A* with the Manhattan Distance heuristic")
        
        # Using time to calculate the search run time
        start = time.time()
        result = search(board, goal, choice)
        end = time.time()
        timeDiff = end - start
        
        # Store result nodes expand, node depth, and max queue size.
        expanded = result.expanded
        depth = result.depth
        maxqsize = result.max_queue_size
        path = [] # Store path taken and revert, then print out the board in order of depth.
        if choice == 1: # Uniform Cost
            while(result.parent != None): # Traverse the tree from leaf to root node
                path.append((result.getBoard(), result.depth, 0)) # Store the board, its depth at each step, and the cost of expanding the node.
                result = result.parent 
            path.append((result.getBoard(), result.depth, 0))
            path.reverse() # Revert the tree to see how Root node's path to leaf node.
        if choice == 2 or choice == 3:
            while(result.parent != None): # Misplaced tile / Manhattan distance 
                path.append((result.getBoard(), result.depth, result.expanded))
                result = result.parent
            path.append((result.getBoard(), result.depth, result.expanded))
            path.reverse()
        
        for i in path: # Print out the board, depth, and cost in each step of the path.
            print("The best state to expand with a g(n) = " + str(i[1]) + " h(n) = " + str(i[2]) + " is... ")
            printBoard(i[0])
        
        print("Solution depth: " + str(depth) + "\n" + 
              "Number of nodes expanded: " + str(expanded) + "\n" + 
              "Max queue size: " + str(maxqsize))
    
    else:
        print("Invalid Selection, program will now exit")
        return 0
    print(f"The algorithm took {timeDiff} seconds to run\n")

main()

Welcome to an 8-Puzzle Solver. Type '1' to use a default puzzle Type '2' to create your own.
1
You wish to use the default puzzle.
Type '1' for Depth 0 
Type '2' for Depth 2 
Type '3' for Depth 4 
Type '4' for Depth 8 
Type '5' for Depth 16 
Type '6' for Depth 20 
Type '7' for Depth 24. 

6
You selected Depth 16
[1 6 7]
[5 0 3]
[4 8 2]
Enter your choice of algorithm: Type '1' for Uniform Cost Search. Type '2' for A* with the Misplaced Tile heuristic. Type '3' for A* with the Manhattan distance heuristic.
1

You selected Uniform Cost Search
The best state to expand with a g(n) = 0 h(n) = 0 is... 
[1 6 7]
[5 0 3]
[4 8 2]
The best state to expand with a g(n) = 1 h(n) = 0 is... 
[1 6 7]
[5 3 0]
[4 8 2]
The best state to expand with a g(n) = 2 h(n) = 0 is... 
[1 6 0]
[5 3 7]
[4 8 2]
The best state to expand with a g(n) = 3 h(n) = 0 is... 
[1 0 6]
[5 3 7]
[4 8 2]
The best state to expand with a g(n) = 4 h(n) = 0 is... 
[1 3 6]
[5 0 7]
[4 8 2]
The best state to expand with a g(n) = 5 h(n) = 0